In [31]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.common.exceptions import NoSuchElementException, TimeoutException
from bs4 import BeautifulSoup
import pandas as pd
import time
import random

def human_delay():
    time.sleep(random.uniform(1, 2)) 

def go_to_next_page():
    try:
        # Wait for the "Next" button to be clickable
        next_button = WebDriverWait(driver, 10).until(
            EC.element_to_be_clickable((By.CSS_SELECTOR, 'a[data-testid="nextButton"]'))
        )
        # Scroll to the "Next" button
        driver.execute_script("arguments[0].scrollIntoView(true);", next_button)
        # Use JavaScript to click the "Next" button
        driver.execute_script("arguments[0].click();", next_button)
        # Wait for the new page to load
        time.sleep(10)  # Adjust sleep time as needed
        # Extract and print the translated text of the next page
        translated_text = driver.find_element(By.TAG_NAME, 'body').text
        print(translated_text)
    except Exception as e:
        print("No more pages or an error occurred:", e)

options = webdriver.ChromeOptions()

# Set preferences to enable translation from Hebrew to English
prefs = {
  "translate_whitelists": {"he": "en"},
  "translate": {"enabled": "true"}
}
options.add_experimental_option("prefs", prefs)

# Set headless mode to True to run without opening a browser window
options.headless = True
options.add_argument("user-agent=Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/91.0.4472.124 Safari/537.36")

# Initialize the WebDriver with the specified options
driver = webdriver.Chrome(options=options)

data_inc = {'car_name': [], 'car_info': [], 'car_year_hand': [], 'car_price': [],'car_tag': []}
url = f"https://www.yad2.co.il/vehicles/cars?price=1000--1&priceOnly=1&imgOnly=1"
driver.get(url)
for i in range(1, 1412):
    try:
        WebDriverWait(driver, 10).until(EC.presence_of_element_located((By.TAG_NAME, "body")))  # Simple condition
    except TimeoutException:
        print("Timed out waiting for page to load: ", url)
        continue
    # Scroll to the bottom of the page (if needed)
    # Scroll down to the bottom of the page
    driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")
    human_delay()  # Allow extra time for any lazy-loaded elements

    # Random scroll up and down
    for _ in range(random.randint(1, 2)):
        scroll_height = random.randint(100, 1500)
        driver.execute_script(f"window.scrollBy(0, -{scroll_height});")
        human_delay()
        driver.execute_script(f"window.scrollBy(0, {scroll_height});")
        human_delay()
        
    # Extract HTML content
    html_content = driver.page_source
    soup = BeautifulSoup(html_content, "html.parser")
    car_boxes = soup.find_all('div', class_='feed-item-base_feedItemBox__5WVY1 scale-up-on-hover_scaleUpOnHover__QSVDw')
    for box in car_boxes:
        car_name = box.find('span', class_='feed-item-info_heading__k5pVC').get_text(strip=True)
        car_info = box.find('span', class_='feed-item-info_marketingText__eNE4R').get_text(strip=True)
        car_year_hand = box.find('span', class_='feed-item-info_yearAndHandBox___JLbc').get_text(strip=True)
        car_price = box.find('span', class_='price_price__xQt90').get_text(strip=True)
        car_tags = box.find('div', class_='tag-list_upgradeItemFlagBox__qPadL')
        if car_tags:
            car_tags_texts = [tag.get_text(strip=True) for tag in car_tags.find_all('span', class_='tag-list_tag__EX6gE flag_baseFlag__PIHA5 flag_grayFlag__1s_oj')]
        else:
            car_tags_texts = ['']

        data_inc['car_name'].append(car_name)
        data_inc['car_info'].append(car_info)
        data_inc['car_year_hand'].append(car_year_hand)
        data_inc['car_price'].append(car_price)
        data_inc['car_tag'].append(','.join(car_tags_texts))

        # Call the function to navigate to the next page
    go_to_next_page()
   
# Close the driver after the loop
driver.quit()# Display the first few rows of the DataFrame


Please note: this site operates a click-accessible system that aids the site's accessibility. Press Control-F11 to adapt the site for the blind who use screen reader software; Press Control-F10 to open the Accessibility menu.
accessibility
נדל״ן
vehicle
Second hand
IL needed
Businesses for sale
Pets
Professionals
Second hand magazine
connection
Posting an ad
private and commercial
motorcycles
scooters
trucks
vessel
special
car type
manufacturer
model
sleep
Additional filters2
Search
 Create an alert
Vehicles for sale
56,443 results
Sort by date
Chevrolet Trax
LT Luxury auto 1.4 (140 hp)
2019 • 2nd hand
69,000 ₪
Distance controlMagnesium wheelsTest in force
Citroen Berlingo 3rd generation (2018 onwards)
Shine OT diesel 7 mk 1.5 (130 hp)
2020 • 1st hand
79,000 ₪
Distance controlEconomicalTest in force
Lexus RX350
4X4 Premium Auto 3.5 (276 HP)
2007 • Hand 5
25,000 ₪
Magnesium wheelsPowerful engineTest in force
Nissan Qashqai
Acenta auto 1.2 (115 hp)
2015 • 3rd hand
43,375 ₪
EconomicalMagn

KeyboardInterrupt: 

In [32]:
df_all_possible_cars = pd.DataFrame(data_inc)
df_all_possible_cars

,car_name,car_info,car_year_hand,car_price,car_tag
0,Mercedes A-Class,A250 Exc Plus hybrid hatchback 1.3 (160 hp),2022 • 1st hand,"210,000 ₪","Hybrid,Test in force,first hand"
1,BMW 7 Series,740LE IPer Luxury Hybrid Auto 2.0 (258 HP),2019 • Hand 1,"284,700 ₪","Adaptive cruise control,Distance control,Hybrid"
2,Alfa Romeo Giulia,Milano Auto 2.0 (197 hp),2018 • 1st hand,"92,400 ₪","Distance control,Economical,Magnesium wheels"
3,Mercedes S-Class,S350D Long Vision OT Diesel 3.0 (258 HP),2017 • 2nd hand,"285,900 ₪","Distance control,Economical,Magnesium wheels"
4,BMW 3 Series,318I Business Auto 2.0 (156 HP),2024 • 1st hand,"299,000 ₪","Distance control,Magnesium wheels,Test in force"
...,...,...,...,...,...
10075,Audi E-tron Q4,Teck Pack electric auto (204 hp),2023 • 1st hand,"291,000 NIS",
10076,Hyundai i30CW,Inspire station wagon 1.6 (126 hp),2010 • 4th hand,"14,900 NIS",
10077,Peugeot 5008,Active Pack automatic 1.2 (130 hp),2024 • 1st hand,"169,900 NIS",
10078,Mercedes GLE Coupe,4X4 GLE350DE AMG Plus automatic 2.0 (194 hp) K3,2021 • 1st hand,"429,000 NIS",


In [33]:
df_all_possible_cars.to_csv('09-06-2024.csv')